# <center>DLRM model</center>

### Import depencies

In [1]:
# !pip install fbgemm-gpu
# !pip install fbgemm-gpu-cpu
# !pip install torch
# !pip install torchrec
# !pip install torchrec-cpu
# !pip install pyre_extensions

In [2]:
import pandas as pd
import numpy as np
import random
import torch     
from sklearn.model_selection import train_test_split
#import custom function from book_funcs file
from book_funcs import category_cleaner, df_cleaner, category_cleaner,
category_compliter, zero_droper, precentage_null

### Import Data

In [3]:
df = pd.read_csv(r'/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv')

In [4]:
df['age'] = df['age'].astype(int)
df['year_of_publication'] = df['year_of_publication'].astype(int)

### train & test split

In [5]:
train_df, test_df = train_test_split(
    df, test_size=0.33, random_state=42)

### Apply custom transformers

In [6]:
train_df = category_cleaner(train_df)
train_df = df_cleaner(train_df)
train_df = category_compliter(train_df)
train_df = zero_droper(train_df)

The original data frame shape:	(690887, 19)
The data frame shape after bookk filtering:	(376474, 19)
The new data frame shape:	(340545, 19)


/Users/royzalta/Documents/GitHub/books_recom_sys/book_funcs.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_author9_relevant_two_options['pred_Category'] = df_author9_relevant_two_options['Category'].apply(


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121040 entries, 0 to 121039
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   user_id              121040 non-null  int64 
 1   location             121040 non-null  object
 2   age                  121040 non-null  int64 
 3   isbn                 121040 non-null  object
 4   rating               121040 non-null  int64 
 5   book_title           121040 non-null  object
 6   book_author          121040 non-null  object
 7   year_of_publication  121040 non-null  int64 
 8   publisher            121040 non-null  object
 9   img_s                121040 non-null  object
 10  img_m                121040 non-null  object
 11  img_l                121040 non-null  object
 12  Summary              121040 non-null  object
 13  Language             121040 non-null  object
 14  Category             121040 non-null  object
 15  city                 118306 non-nu

In [12]:
#create a function to check for nulls 
def precentage_null(df):
    null_count = df.isnull().sum()
    total_count = len(df)
    print((null_count/total_count) * 100)

In [13]:
null_train = precentage_null(train_df)
print(null_train)

user_id                0.000000
location               0.000000
age                    0.000000
isbn                   0.000000
rating                 0.000000
book_title             0.000000
book_author            0.000000
year_of_publication    0.000000
publisher              0.000000
img_s                  0.000000
img_m                  0.000000
img_l                  0.000000
Summary                0.000000
Language               0.000000
Category               0.000000
city                   2.258757
state                  3.170026
country                5.074356
location_enc           0.000000
publisher_enc          0.000000
Language_enc           0.000000
Category_enc           0.000000
dtype: float64
None


### Encoding sparse features

In [8]:
cols_dense = ["age", "year_of_publication"]
cols_sparse = ['location',
               'publisher','Language','Category',
              'city','state','country']

In [9]:
def encode(col, map_rev):
    encoded_values = []
    for item in col:
        if pd.notna(item):  # Check if the value is not NaN
            encoded_value = map_rev.get(item, item)
            encoded_values.append(encoded_value)
        else:
            # Handle NaN values as needed
            encoded_values.append('NaN')
    return encoded_values


In [10]:
map_sparse = {}
map_sparse_rev = {}

In [11]:
for feat in cols_sparse:
    map_sparse[feat] = {i: c for i,c in enumerate(train_df[feat].explode().value_counts().index)}
    map_sparse_rev[feat] = {v:k for k, v in map_sparse[feat].items()}

    train_df[feat+'_enc'] = train_df[feat].apply(lambda x: encode(x, map_sparse_rev[feat]))
    test_df[feat+'_enc'] = test_df[feat].apply(lambda x: encode(x, map_sparse_rev[feat]))


TypeError: 'float' object is not iterable